# Building a dataset for NREL and LCD weather and solar data
This notebook is intended to be a first step in gathering data for a solar energy forecasting model. The final dataset will consist of hourly weather data and matching hourly solar data for multiple weather stations. This dataset can then be used for analytical analysis or to train a machine learning model.

## LCD data acquisition
We are going to first grab data from the [Local Climatological Data (LCD)](https://www.ncdc.noaa.gov/data-access/land-based-station-data/land-based-datasets/quality-controlled-local-climatological-data-qclcd) Dataset.
LCD dataset is a NCEI NOAA open database. It contains historic hourly weather data
for major weather stations in the US. Primarily what we are looking for is cloud coverage data to compare with the solar data later on. There are many other public and private datasets that could have been used instead of LCD, but this one is free and open to anyone. For more accurate data, a paid historical dataset could have been used.

To begin, first we need to set the parameter variables for selecting our data.

In [108]:
year = '2017' # 2017 is the max year
states = ['Nevada', 'Texas', 'Arizona', 'New Mexico'] # Use full capitalized names (any US state)
station_count = None # None = all stations

Now we can set up the url path and the station metadata that will be used after to extract the raw data using Pandas.

In [109]:
import pandas as pd

data_path = 'https://www.ncei.noaa.gov/data/local-climatological-data/access/'+ year +'/'
stations = pd.read_csv('stations.csv', dtype={"LATITUDE": float, "LONGITUDE": float})
stations = stations[stations['STATE'].isin(states)][:station_count]

print(stations.shape)
stations.head()

(278, 10)


,file,STATION_ID,STATION,BEGIN_DATE,END_DATE,STATE,COUNTRY,LATITUDE,LONGITUDE,ELEVATION_(M)
1,72011053983.csv,53983,"LLANO MUNICIPAL AIRPORT, TX US",2004-12-31,2020-06-21,Texas,United States,30.78361,-98.66194,335.9
5,72015103049.csv,3049,"ALPINE CASPARIS MUNICIPAL AIRPORT, TX US",2004-12-31,2020-06-21,Texas,United States,30.38333,-103.68333,1375.6
16,72026153976.csv,53976,"BRIDGEPORT MUNICIPAL AIRPORT, TX US",2004-12-31,2020-06-21,Texas,United States,33.17528,-97.82833,259.4
22,72026912982.csv,12982,"FALFURRIAS BROOKS CO AIRPORT, TX US",2004-12-31,2020-05-15,Texas,United States,27.20667,-98.12111,34.1
23,72027103044.csv,3044,"BIG SPRING MCMAHON WRINKLE AIRPORT, TX US",2004-12-31,2020-06-21,Texas,United States,32.21250,-101.52139,784.3


Before we get the data, we will create a lambda expression that will clean up floating point numbers by removing unwanted characters.

In [110]:
clean_float = lambda x: ''.join(filter(lambda y: y.isdigit() or y == '.', x))
clean_float('23.5s')

'23.5'

Next we will import tqdm. This library adds loading bar functionality
which will help visualize some of the larger Pandas operations.

In [111]:
from tqdm.auto import tqdm
tqdm.pandas()

/Users/gagekrumbach/.local/share/virtualenvs/Solar_Forcaster-m3btlaz4/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


Finally we can access our data through the LCD database using HTTPS. We only extract a set amount of columns that we need. We also pass in our float cleaner for three of the columns.

In [112]:
from datetime import datetime as dt

w_data = pd.concat([pd.read_csv(data_path + station[1]['file'],
                            usecols=['STATION', 'DATE', 'HourlySkyConditions', 'HourlyStationPressure',
                                'HourlyVisibility', 'HourlyPresentWeatherType', 'HourlyPrecipitation'],
                            parse_dates=['DATE'],
                            dtype={'HourlySkyConditions': str, 'HourlyPresentWeatherType': str},
                            converters={'HourlyStationPressure': clean_float, 'HourlyVisibility': clean_float,
                                        'HourlyPrecipitation': clean_float})
                         for station in tqdm(stations.iterrows(), total=stations.shape[0])\
                      if dt.strptime(station[1]["BEGIN_DATE"], '%Y-%m-%d') < dt.strptime(year, '%Y')\
                      and dt.strptime(station[1]["END_DATE"], '%Y-%m-%d') > dt.strptime(year, '%Y')])

 

## Data manipulation with Pandas

Now we will convert weather codes (ex: `-SN:03 BR:1 |SN |`) to a less specific form that can be used by the ML model later on. More information on these codes can be found [here](https://www.ncei.noaa.gov/data/local-climatological-data/doc/LCD_documentation.pdf). Codes are in the format `AU | AW | MW` with each observation separated by a bar. They need to be categorized into `freezing_rain_heavy, freezing_rainice_pellets, ice_pellets_light, tstorm` and other categories. We will perform this operation by using a json lookup file.

In [113]:
import json
with open('weather_lookup_converter.json') as json_file:
    lookup = json.load(json_file)

def weather_decoder(value):
    if pd.notnull(value):
        for weather_type in lookup:
            for code in lookup[weather_type]:
                if code in value:
                    return weather_type

# apply the decoder using a lambda expression
w_data['weather_type'] = w_data.progress_apply(lambda row: weather_decoder(row.HourlyPresentWeatherType), axis=1)

Next we do the same manipulation but with cloud observations (ex: `OVC:08 90`). We break up the hourly sky conditions column to a cloud string and cloud cover percentage using another lookup table. These will be categorized into `cloudy, partly_cloudy, clear` and other categories.

In [114]:
with open('cloud_lookup.json') as json_file:
    lookup = json.load(json_file)

def cloud_decoder(value):
    if pd.notnull(value):
        for cloud_type in lookup:
            if cloud_type in value:
                return lookup[cloud_type]
    return lookup["CLR"]
            
w_data['cloud_type'] = w_data.progress_apply(lambda row: cloud_decoder(row.HourlySkyConditions)["cloud_str"], axis=1)
w_data['cloud_cover'] = w_data.progress_apply(lambda row: cloud_decoder(row.HourlySkyConditions)["cloud_cover"], axis=1)

w_data.head()

,STATION,DATE,HourlyPrecipitation,HourlyPresentWeatherType,HourlySkyConditions,HourlyStationPressure,HourlyVisibility,weather_type,cloud_type,cloud_cover
0,72011053983,2017-01-01 00:15:00,,NaN,CLR:00,28.71,10.00,None,clear,0.0
1,72011053983,2017-01-01 00:35:00,,NaN,CLR:00,28.71,10.00,None,clear,0.0
2,72011053983,2017-01-01 00:55:00,,NaN,CLR:00,28.71,10.00,None,clear,0.0
3,72011053983,2017-01-01 01:15:00,,NaN,CLR:00,28.71,10.00,None,clear,0.0
4,72011053983,2017-01-01 01:35:00,,NaN,CLR:00,28.70,10.00,None,clear,0.0


## Combine meta data and actual data
Next we will combine the station metadata from before (lat, long, elevation) with the historic weather data. Before we do this, we need to alter the historic weather data station ids so they match the metadata station ids.<br/><br/>
The metadata uses a 5 digit WBAN id: `54852` and the historic data uses a 6 digit USAF MASTER id + the WBAN id: `72034354852`.

In [115]:
# remove USAF MASTER id
w_data['STATION'] = w_data['STATION'].map(lambda x: int(str(x)[-5:]))


# add the meta data to the weather data
w_data["latitude"] = w_data.progress_apply(lambda row:
                                      float(stations[stations["STATION_ID"] == row.STATION]['LATITUDE']), axis=1)
w_data["longitude"] = w_data.progress_apply(lambda row:
                                      float(stations[stations["STATION_ID"] == row.STATION]['LONGITUDE']), axis=1)
w_data["elevation"] = w_data.progress_apply(lambda row:
                                      float(stations[stations["STATION_ID"] == row.STATION]['ELEVATION_(M)']), axis=1)
w_data.head()

,STATION,DATE,HourlyPrecipitation,HourlyPresentWeatherType,HourlySkyConditions,HourlyStationPressure,HourlyVisibility,weather_type,cloud_type,cloud_cover,latitude,longitude,elevation
0,53983,2017-01-01 00:15:00,,NaN,CLR:00,28.71,10.00,None,clear,0.0,30.78361,-98.66194,335.9
1,53983,2017-01-01 00:35:00,,NaN,CLR:00,28.71,10.00,None,clear,0.0,30.78361,-98.66194,335.9
2,53983,2017-01-01 00:55:00,,NaN,CLR:00,28.71,10.00,None,clear,0.0,30.78361,-98.66194,335.9
3,53983,2017-01-01 01:15:00,,NaN,CLR:00,28.71,10.00,None,clear,0.0,30.78361,-98.66194,335.9
4,53983,2017-01-01 01:35:00,,NaN,CLR:00,28.70,10.00,None,clear,0.0,30.78361,-98.66194,335.9


## Solar data acquisition
Now we will load the [NREL](https://nsrdb.nrel.gov/) data from the National Solar Radiation Database (NSREL). We use their api to get solar data for each station in our meta data. This will later be merged together to form a complete dataset.<br/><br/>
We will be using the Physical Solar Model ([PSM](https://nsrdb.nrel.gov/about/u-s-data.html#psm)) v3 that NSREL provides. To get an api key, follow the guide [here](https://developer.nrel.gov/docs/solar/nsrdb/psm3_data_download/), but the api key `DEMO_KEY` will also work fine. Keep in mind that this key only has a rate limit of 30 calls a day per IP address.

In [116]:
# set api key (use 'DEMO_KEY' if not set up)
key = 'DEMO_KEY'

import os
key = os.getenv("MY_VAR")

# trim dataset to match DEMO_KEY limits
if(key == 'DEMO_KEY'):
    w_data = w_data[w_data["STATION"].isin(w_data['STATION'].unique()[:3])]
    print("Stations: " + str(w_data['STATION'].unique()))

Because the api only allows us to get one location per call, we will need to complete a series of calls. First we will create a function to create the urls.

In [117]:
def create_url(lat, lon, year, api_key, attributes, your_name, reason_for_use, your_affiliation, your_email):
    return 'https://developer.nrel.gov/api/solar/nsrdb_psm3_download.csv?wkt=POINT({lon}%20{lat})&names={year}&leap_day=false&interval=30&utc=false&full_name={name}&email={email}&affiliation={affiliation}&mailing_list=false&reason={reason}&api_key={api}&attributes={attributes}'\
    .format(year=year, lat=lat, lon=lon,
        name=your_name, email=your_email,
        affiliation=your_affiliation,
        reason=reason_for_use, api=api_key,
        attributes=attributes)

Now we will iterate through each station in our data by using Pandas' `.unique()` function. Then we will form the url and append the data into one list.

In [118]:
import time
for step, point in enumerate(tqdm(stations[stations["STATION_ID"].isin(w_data['STATION'].unique())].iterrows(),
                                  total=len(w_data['STATION'].unique()))):
    url = create_url(lat=str(point[1][7]),
                     lon=str(point[1][8]),
                     api_key=key,
                     attributes="air_temperature,dew_point,dhi,dni,ghi,relative_humidity,wind_direction,wind_speed,clearsky_dhi,clearsky_dni,clearsky_ghi",
                     year=year,
                     your_name='Gage+Krumbach',
                     reason_for_use='demo',
                     your_affiliation='my+institution',
                     your_email='gkrumbac@redhat.com')
    
    output = pd.read_csv(url, header=2)
    
    output["STATION"] = int(point[1][1])
    output["DATE"] =pd.date_range('1/1/{yr}'.format(yr=year),
                                                    freq='30'+'Min',
                                                    periods=525600/60)
    if step==0:
        solar_data = output
    else:
        solar_data = solar_data.append(output)
    
    # we wait here because the api can't be called more than once in 1 seconds
    time.sleep(1)

solar_data.head()

,Year,Month,Day,Hour,Minute,Temperature,Dew Point,DHI,DNI,GHI,Relative Humidity,Wind Direction,Wind Speed,Clearsky DHI,Clearsky DNI,Clearsky GHI,STATION,DATE
0,2017,1,1,0,30,11,8,0,0,0,87.97,4.0,1.0,0,0,0,53983,2017-01-01 00:00:00
1,2017,1,1,1,30,10,7,0,0,0,89.77,3.6,0.9,0,0,0,53983,2017-01-01 01:00:00
2,2017,1,1,2,30,9,6,0,0,0,85.90,6.4,0.8,0,0,0,53983,2017-01-01 02:00:00
3,2017,1,1,3,30,8,6,0,0,0,88.00,3.6,0.8,0,0,0,53983,2017-01-01 03:00:00
4,2017,1,1,4,30,8,5,0,0,0,90.96,347.9,0.8,0,0,0,53983,2017-01-01 04:00:00


## Merge solar data and weather data
Finally we can merge the solar data and weather data. We do this by using Pandas' `merge_asof()` function which will merge two dataframes by the nearest key. Meaning, a row with a date of `1-2-2017 11:34` will merge with a date `1-2-2017 11:55`, because it is the closest.

In [119]:
df_merge_asof = pd.merge_asof(solar_data.sort_values(by=['DATE']), w_data.sort_values(by=['DATE']),
              on='DATE',
              by='STATION',
             direction='nearest')

## Altair visualization
We can use Altair to visualize the weather stations.

In [120]:
import altair as alt
from vega_datasets import data

states = alt.topo_feature(data.us_10m.url, feature='states')

coord_center = [w_data['longitude'].unique().mean(),w_data['latitude'].unique().mean()]

background = alt.Chart(states).mark_geoshape(
    fill='#4D4D4D',
    stroke='white'
).project(
    type= 'mercator',
    scale= 500,
    center= coord_center
).properties(
    title='Stations',
    width=400, height=300
)

st = alt.Chart(stations[stations["STATION_ID"].isin(w_data['STATION'].unique())]
).mark_circle(
    size=45,
    color='white'
).encode(
    longitude='LONGITUDE:Q',
    latitude='LATITUDE:Q',
    tooltip=['STATION:N']
).project(
    type= 'mercator',
    scale= 2000,
    center= coord_center
)

all_st = alt.Chart(pd.read_csv('stations.csv', dtype={"LATITUDE": float, "LONGITUDE": float})
).mark_circle(
    size=10,
    color='#FAB6B6'
).encode(
    longitude='LONGITUDE:Q',
    latitude='LATITUDE:Q'
    ).project(
    type= 'mercator',
    scale= 2000,
    center= coord_center
)

background + all_st + st

alt.LayerChart(...)

To cleanup the data further, we only select the columns that are relevant.

In [124]:
df_merge_asof = df_merge_asof[['STATION', 'DATE', 'latitude', 'longitude', 'elevation',
                               'Temperature', 'Dew Point', 'Relative Humidity', 'HourlyStationPressure',
                               'Wind Direction', 'Wind Speed', 'HourlyVisibility', 'HourlyPrecipitation',
                               'weather_type', 'cloud_type', 'cloud_cover', 'DHI', 'DNI', 'GHI',
                               "Clearsky DHI","Clearsky DNI","Clearsky GHI"
                              ]]
df_merge_asof.head()

,STATION,DATE,latitude,longitude,elevation,Temperature,Dew Point,Relative Humidity,HourlyStationPressure,Wind Direction,Wind Speed,HourlyVisibility,HourlyPrecipitation,weather_type,cloud_type,cloud_cover,DHI,DNI,GHI,Clearsky DHI,Clearsky DNI,Clearsky GHI
0,53983,2017-01-01,30.78361,-98.66194,335.9,11,8,87.97,28.71,4.0,1.0,10.00,,None,clear,0.0,0,0,0,0,0,0
1,53973,2017-01-01,30.87194,-96.62222,86.9,12,11,100.00,29.56,334.9,1.0,4.00,,drizzle,cloudy,0.8,0,0,0,0,0,0
2,3184,2017-01-01,33.68833,-112.08167,443.5,10,8,95.61,28.26,121.7,1.4,10.00,0.00,None,cloudy,0.8,0,0,0,0,0,0
3,23111,2017-01-01,33.55000,-112.36667,330.7,11,8,91.23,28.66,127.3,1.3,7.00,,None,mostly_cloudy,0.6,0,0,0,0,0,0
4,23104,2017-01-01,33.30000,-111.66667,421.2,10,8,94.68,28.41,121.7,1.4,10.00,,None,partly_cloudy,0.3,0,0,0,0,0,0


We take this final data and export it back out as a csv.

In [126]:
df_merge_asof.to_csv("out.csv",index=False)